In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

### Manual Spectral Graph Layer 
##### 1.) N/V --> Number of Vertex in the Graph
##### 2.) E --> Number of Edges in the Graph
##### 3.) W --> Weighted Adjacency Matrix Shape: [N, N]
##### 4.) A signal X: V  --> R defined on the ndes of the graph may be regarded as a vector

##### 5.) An essential operator in spectral graph analysis is the graph Laplacian, which is represented as L = D-W, where each of matrix has a shape of [N, N], 
###### 5.1) D: Node degree matrix [It will be a diagonal matrix representing the number of degree of each node]
###### 5.2) W: This is the weighted adjacency matrix. 

##### 6.) The normalized laplacian matrix is represented as L = I- D^-0.5*W*D^-0.5 where I is the identity matrix. 

##### 7.) L has a complete set of orthonormal eigenvectors


In [ ]:
class ManualSpectralGraphLayer(nn.Module):

    def __init__(self, input_dim, output_dim, k = 0, bias = True):
        super(ManualSpectralGraphLayer, self).__init__()
        self.k = k
        self.weight = Parameter(torch.FloatTensor(input_dim, output_dim))
        # K represents the number of Hops 
        self.bias= bias
        if self.bias == True:
            self.bias = Parameter(torch.FloatTensor(output_dim))
    
    def tilda(self, mat, max):
        mat = mat/max - torch.eye(mat.shape[0])
        return mat
    
    def laplace(self, adj):
        # this function calculates the normalized laplacian matrix. 
        degree_mat = torch.sum(adj, dim = 1)
        eye_ = torch.eye(adj.shape[0])
        adj_norm = eye_ - degree_mat.pow(-0.5)@adj@degree_mat.pow(-0.5)
        return adj_norm

    def forward(self, adj):

        output_torch = torch.empty([self.k, adj.shape[0], adj.shape[0]])
        eigenval, eigenvec = torch.linalg.eig(adj)
        max_eigen_val = max(eigenval)
        if self.k == 0:
            # In case if k = 0, then direct input has to be fed in the output_torch
            output_torch[0] = adj
        elif self.k == 1:
            # in case if k= 1, then direct input and Lx
            output_torch[0] = adj
            output_torch[1] = self.laplace(adj) @ adj
        elif self.k > 1:
            output_torch[0] = adj
            output_torch[1] = self.laplace(adj) @ adj
            for i in range(2, self.k):
              output_torch[i] = 2 * self.laplace(adj) @ output_torch[i-1] - output_torch[i-2]

        output = output_torch @ self.weight
        if self.bias :
            output = output + self.bias
        
        return output
        

In [5]:
torch.empty([1,3])

tensor([[-1.1215e+36,  4.5857e-41, -1.1215e+36]])